In [1]:
### Intialize Main ###
def FEA_prototype(Text_file):
    import numpy as np
    import csv
    f = open(Text_file)
    reader = csv.reader(f, delimiter=' ')
    data_string = []
    for row in reader:
        data_string.append(row) # in a string form
    # read Mesh
    node_num = []
    x_distance = []
    for i in range(1, int(data_string[0][0]) + 1):
        node_num.append(int(data_string[i][0])-1)
        x_distance.append(float(data_string[i][1]))
    # read properties
    element_num = []
    connectivity_of_element = []
    for i in range(int(data_string[0][0]) + 1 , int(data_string[0][0]) + int(data_string[0][1]) + 1):
        element_num.append(float(data_string[i][0]))
        connectivity_of_element.append((int(data_string[i][1])-1,int(data_string[i][2])-1))
    for i in range(int(data_string[0][0]) + int(data_string[0][1]) + 1, int(data_string[0][0]) + int(data_string[0][1]) + 2):
        E = float(data_string[i][0])
        height = float(data_string[i][1])
        width = float(data_string[i][2])
    # read constraints
    num_of_constraints = int(data_string[int(data_string[0][0]) + int(data_string[0][1]) + 2][0])
    dof_to_be_zeros = []
    for i in range(num_of_constraints):
        dof_to_be_zeros.append(int(data_string[int(data_string[0][0]) + int(data_string[0][1]) + 3][i])-1)
    # read loads
    number_of_point_loads = int(data_string[len(data_string) - 2][0])
    load_dof = int(data_string[len(data_string) - 1][0]) - 1
    load = float(data_string[len(data_string) - 1][1])
    # get Element DOF
    def get_element_dof(element_connection):
        node_1,node_2 = int(element_connection[0]),int(element_connection[1])
        dof_node = node_1*2,node_1*2+1,node_2*2,node_2*2+1
        return dof_node
    # get Element K
    def get_element_k(nodes_that_are_connnected):
        EI = E * (1/12)*width * height**3
        x,y = nodes_that_are_connnected
        h = abs(x_distance[x]-x_distance[y]) # difference in x
        K = EI * np.array([[12/h**3, 6/h**2, -12/h**3, 6/h**2], [6/h**2, 4/h, -6/h**2, 2/h], [-12/h**3, -6/h**2, 12/h**3, -6/h**2], [6/h**2, 2/h, -6/h**2, 4/h]])
        return K
    # assemble Global Stiffness Matrix
    def global_K(connectivity):
        K = np.zeros((int(len(node_num)*2), int(len(node_num)*2)))
        for i in connectivity:
            dof_0 = get_element_dof(i)[0]
            dof_1 = get_element_dof(i)[1]
            dof_2 = get_element_dof(i)[2]
            dof_3 = get_element_dof(i)[3]
            K[dof_0][dof_0] += get_element_k(i)[0][0]
            K[dof_0][dof_1] += get_element_k(i)[0][1]
            K[dof_0][dof_2] += get_element_k(i)[0][2]
            K[dof_0][dof_3] += get_element_k(i)[0][3]
            
            K[dof_1][dof_0] += get_element_k(i)[1][0]
            K[dof_1][dof_1] += get_element_k(i)[1][1]
            K[dof_1][dof_2] += get_element_k(i)[1][2]
            K[dof_1][dof_3] += get_element_k(i)[1][3]
            
            K[dof_2][dof_0] += get_element_k(i)[2][0]
            K[dof_2][dof_1] += get_element_k(i)[2][1]
            K[dof_2][dof_2] += get_element_k(i)[2][2]
            K[dof_2][dof_3] += get_element_k(i)[2][3]
            
            K[dof_3][dof_0] += get_element_k(i)[3][0]
            K[dof_3][dof_1] += get_element_k(i)[3][1]
            K[dof_3][dof_2] += get_element_k(i)[3][2]
            K[dof_3][dof_3] += get_element_k(i)[3][3]
        return K
    def F_eq(point_loads):
        f = np.zeros((len(node_num)*2,1))
        f[load_dof] = point_loads
        return f
    # applying kinematic constraints
    final_K = global_K(connectivity_of_element)
    final_feq = F_eq(load)
    for i in dof_to_be_zeros:
        final_K[i][:] = 0
        final_K[:][i] = 0
        final_K[i][i] = 1
        final_feq[i] = 0
    final_displacements = np.linalg.inv(final_K)@final_feq
    echo_print_output_information = [node_num,x_distance,element_num,connectivity_of_element,E,height,width,num_of_constraints,dof_to_be_zeros,number_of_point_loads,load_dof,load]
        # only returning Part 1 in python numbering system
    output_text_file = open("Output_{}.txt".format(Text_file), "w")
    output_text_file.write("#################################\n"+
                            "Benjamin Tollison FEA output file\n"+
                            "#################################\n"+
                            "Number of nodes used: {}\n".format(len(node_num))+
                            "Node list: {}\n".format(node_num)+
                            "x position of nodes: {}\n".format(x_distance)+
                            "Number of Elements: {}\n".format(len(element_num))+
                            "Connection of Elements: {}\n".format(connectivity_of_element)+
                            "Young's Modulus, height, and width of rod: {}, {}, {}\n".format(E,height,width)+
                            "Number of constraints: {}\n".format(num_of_constraints)+
                            "Degrees of freedom to be zero: {}\n".format(dof_to_be_zeros)+
                            "Number of point loads: {}\n".format(number_of_point_loads)+
                            "DOF where load is: {}\n".format(load_dof)+
                            "Load: {}\n".format(load)+
                            "Nodal Displacements: \n"+
                            "1: v = {}, m = {},\n".format(np.round(final_displacements[0],7),np.round(final_displacements[1],7))+
                            "2: v = {}, m = {},\n".format(np.round(final_displacements[2],7),np.round(final_displacements[3],7))+
                            "3: v = {}, m = {},\n".format(np.round(final_displacements[4],7),np.round(final_displacements[5],7))+
                            "4: v = {}, m = {},\n".format(np.round(final_displacements[6],7),np.around(final_displacements[7],7))
                            )
    output_text_file.close()
    f.close()
    return print(final_displacements)

FEA_prototype('data_1.txt')

[[-6.98386079e-20]
 [-1.42814226e-04]
 [ 2.10200464e-19]
 [-1.18836433e-19]
 [ 3.09574393e-19]
 [ 3.57035565e-05]
 [ 3.09430823e-03]
 [-3.92739122e-04]]
